# Data Wrangling Part 1

## Data Formatting part 1

### Correcting Datatypes
Converting the datatype to correct format by exploring.
- Identify the datatypes of each column
    ```python
    print(dataframe.dtypes)
    ```
- Converting necessary datatypes. We can use float, float32, float64, int, int32, int64, str. We can prevent the data loss by taking the higher precision datatype __64
    ```python
    dataframe[float_column_names] = dataframe[float_column_names].astype("float64")
    dataframe[int_column_names] = dataframe[int_column_names].astype("int64")
    dataframe[str_column_names] = dataframe[str_column_names].astype("str")
    print(dataframe.dtypes)
    ```

In [1]:
def correct_datatype(df, float_column_names, int_column_names, str_column_names):
    print(df.dtypes)
    
    df[float_column_names] = df[float_column_names].astype("float64")
    df[int_column_names] = df[int_column_names].astype("int64")
    df[str_column_names] = df[str_column_names].astype("str")
    print(df.dtypes)
    
    return df

### Maintaining Data Consistency
#### String Cleaning
- strip() method of a string removes leading and trailing whitespace (spaces, tabs, and newline characters) from a string. You can specify the characters you want to remove by providing a regular expression to the method. The argument must contains all the characters that are not allowed in leading or trailing. It doesn't affect within the middle of the string. 
- lstrip() method works stripping on leading strings
- rsrtip() method works on trailing strings
    ```python
    dataframe[column_names] = dataframe[column_names].str.strip("1234567890!@#$%^&*()_+=<>?,./;:'\"[]{}\\|-`~")
    dataframe[column_names] = dataframe[column_names].str.lstrip()
    dataframe[column_names] = dataframe[column_names].str.rstrip()
    ```
- standarizing a string needs to first remove something or replace with '' empty string and then replace with standard form
    ```python
    # [^] means anything except [] to replace with empty 
    dataframe[column] = dataframe[column].str.replace('[^a-zA-Z0-9]', '')
    
    # standarizing Example
    dataframe[column] = dataframe[column].apply(lambda x: str(x))
    dataframe[column] = dataframe[column].apply(lambda x: x[0:3] + '-' + x[3:6] + '-' + x[6:10])
    ```
- Standarizing depends on us completely. We have to carefully do everything

- We can split a value into columns if necessary
    ```python
    # 1st argument is to tell when to split, 2nd is the number of splits
    dataframe[col_names] = dataframe[col_to_split].str.split(',',2, expand=True)
    ```
- We can correct typos or replace withing string

- Character encodings are specific sets of rules for mapping from raw binary byte strings (that look like this: 0110100001101001) to characters that make up human-readable text (like "hi"). You might also end up with a "unknown" characters. There are what gets printed when there's no mapping between a particular byte and a character in the encoding you're using to read your byte string in and looking like ???.. UTF-8 is the standard text encoding. All Python code is in UTF-8 and, ideally, all your data should be as well. It's when things aren't in UTF-8 that you run into trouble. If you look at a bytes object, you'll see that it has a b in front of it, and then maybe some text after. That's because bytes are printed out as if they were characters encoded in ASCII. 
It was pretty hard to deal with encodings in Python 2, but thankfully in Python 3 it's a lot simpler. (Kaggle Notebooks only use Python 3.) There are two main data types you'll encounter when working with text in Python 3. One is is the string, other one is bytes.
however, any characters not in ASCII will just be replaced with the unknown character. Then, when we convert the bytes back to a string, the character will be replaced with the unknown character. The dangerous part about this is that there's not way to tell which character it should have been. That means we may have just made our data unusable!
    ```python
    # start with a string
    before = "This is the euro symbol: €"

    # encode it to a different encoding, replacing characters that raise errors
    after = before.encode("ascii", errors = "replace")

    # convert it back to utf-8
    print(after.decode("ascii"))

    # This is the euro symbol: ?
    
    new_entry = (sample_entry.decode("ASCII", errors='replace')).encode("utf-8", errors='replace')
    ```
- Reading files with encoding problems: During reading a csv a file we may encounter UnicodeDecodeError. This tells us that this file isn't actually UTF-8. We don't know what encoding it actually is though. 
1. One way to figure it out is to try and test a bunch of different character encodings and see if any of them work. 
2. A better way, though, is to use the charset_normalizer module to try and automatically guess what the right encoding is. It's not 100% guaranteed to be right, but it's usually faster than just trying to guess. 
3. just look at the first ten thousand bytes of this file. This is usually enough for a good guess about what the encoding is and is much faster than trying to look at the whole file. 

    ```python
    # helpful character encoding module
    import charset_normalizer
    
    # try to read in a file not in UTF-8
    df = pd.read_csv("abcd.csv")
    
    # Error: UnicodeDecodeError                        Traceback (most recent call last)
    
    # look at the first ten thousand bytes to guess the character encoding
    with open("abcd.csv", 'rb') as rawdata:
        result = charset_normalizer.detect(rawdata.read(10000))

    # check what the character encoding might be
    print(result)
    
    # {'encoding': 'Windows-1252', 'language': 'English', 'confidence': 0.73} 
    # So charset_normalizer is 73% confidence that the right encoding is "Windows-1252". Let's see if that's correct:
    
    # read in the file with the encoding detected by charset_normalizer
    df = pd.read_csv("abcd.csv", encoding='Windows-1252')

    # look at the first few lines
    df.head()
    ```
    
- Case Standarizing: Converting every string on upper or lower case
    ```python
    dataframe[column] = dataframe[column].str.lower()
    dataframe[column] = dataframe[column].str.upper()
    ```
- Removing non Alphanumeric characters
    ```python
    dataframe[column] = dataframe[column].str.replace('[^a-zA-Z0-9\s]', '', regex=True)
    ```

#### Date and Time Cleaning
- Making sure that all your dates and times are either a DateTime object or a Unix timestamp (via type coercion). No strings pretending to be a DateTime object
    ```python
    # Check the column's type
    print(dataframe[datetime_column].head())
    #  bottom of the output of head(), you can see that it says that the data type of this column is "object".
    ```
- We should check the date column's string length to confirm length is same. Output of the code indicates length and then the number of occurance of that length
    ```python
    date_lengths = dataframe.column_name.str.len()
    print(date_lengths.value_counts())
    ```
- If there is different length, We can retrieve the indices of those irregularities and check the records
    ```python
    indices = np.where([date_lengths == len_with_irr])[1]
    print('Indices with corrupted data:', indices)
    print(dataframe.loc[indices])
    ```
- For those indices we can manually change the format to given format
    ```python
    for i in indices:
        dataframe.loc[i, 'Date'] = parser.parse(dataframe.loc[i, 'Date'])
        dataframe.loc[i, 'Date'] = dataframe.loc[i, 'Date'].strftime("%m/%d/%Y")
    ```
- convert the date columns to datetime. Check python format in [here](https://strftime.org/). There are lots of possible parts of a date, but the most common are %d for day, %m for month, %y for a two-digit year and %Y for a four digit year. Some examples:
    - 1/17/07 has the format "%m/%d/%y"
    - 17-1-2007 has the format "%d-%m-%Y"
    ```python
    # create a new column, date_parsed, with the parsed dates
    from datetime import datetime
    dataframe['date_parsed'] = pd.to_datetime(dataframe[datetime_column], format="%m/%d/%y")
    # the dtype is datetime64 if we see head
    ```
- Problems with multiple date formats: If we see an error when there are multiple date formats in a single column, you can have pandas try to infer what the right date format should be. 
    ```python
    dataframe['date_parsed'] = pd.to_datetime(dataframe[datetime_column], infer_datetime_format=True)
    ```
There are two big reasons not to always have pandas guess the time format. The first is that pandas won't always been able to figure out the correct date format, especially if someone has gotten creative with data entry. The second is that it's much slower than specifying the exact format of the dates.
- Select the day of the month
    ```python
    dataframe['date_parsed'].dt.day
    ```
- Plot the day: One of the biggest dangers in parsing dates is mixing up the months and days. Hence double checking is needed. To do this, let's plot a histogram of the days of the month. We expect it to have values between 1 and 31
    ```python
    import seaborn as sns
    sns.distplot(dataframe['date_parsed'].dt.day, kde=False, bins=31)
    ```
- DateTime objects are often recorded with or without time zones, which can cause issues. If you are doing region-specific analysis, ensure that DateTime objects are in the correct time zone. If internationalization is not a concern, consider converting all DateTime objects to a specific time zone.
    ```python
    import pytz  # For time zone handling
    dataframe['date_time'] = dataframe['date_time'].apply(lambda x: datetime.fromtimestamp(x, pytz.timezone('US/Eastern')))
    ```

In [3]:
import datetime
import seaborn as sns
import pytz  # For time zone handling

def cleaning_datetime(df, col):
    # Check the column's type
    print(df[col].head())
    
    # Check length with frequency
    date_lengths = df.col.str.len()
    print(date_lengths.value_counts())
    
    # Check irregular length record here
    indices = np.where([date_lengths == len_with_irr])[1]
    print('Indices with corrupted data:', indices)
    print(df.loc[indices])
    
    # Correct irregular length record here
    for i in indices:
        df.loc[i, col] = parser.parse(df.loc[i, col])
        df.loc[i, col] = df.loc[i, col].strftime("%m/%d/%Y")
        
    # Change format
    df['date_parsed'] = pd.to_datetime(df[col], format="%m/%d/%y")
    
    # If multiple format then write this line instead
    df['date_parsed'] = pd.to_datetime(df[col], infer_datetime_format=True)
    
    # Check the column's type
    print(df[col].head())
    
    # Check days by plot if there in 1-30
    sns.distplot(df['date_parsed'].dt.day, kde=False, bins=31)
    
    # Change zone
    df['date_time'] = df['date_time'].apply(lambda x: datetime.fromtimestamp(x, pytz.timezone('US/Eastern')))

#### Fix Inconsistent Data Entries
- To get closer look of a feature we can view row
    ```python
    countries = professors['Country'].unique()
    ```
- sort them alphabetically and then take a closer look
    ```python
    countries.sort()
    countries
    ```
- Handle the upper lower consistencies
- There could be same entry but with different style example: southkorea south korea. use the fuzzywuzzy package to help identify which strings are closest to each other.
Fuzzywuzzy returns a ratio given two strings. The closer the ratio is to 100, the smaller the edit distance between the two strings. Here, we're going to get the ten strings from our list of cities that have the closest distance to "south korea".
    ```python
    # get the top 10 closest matches to "south korea"
    matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # take a look at them
    matches
    ```
output:
[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('austria', 33),
 ('ireland', 33),
 ('pakistan', 32),
 ('portugal', 32),
 ('scotland', 32),
 ('australia', 30)]
 
 We can see that two of the items in the cities are very close to "south korea": "south korea" and "southkorea". Let's replace all rows in our "Country" column that have a ratio of > 47 with "south korea".

In [ ]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match

    
    # let us know the function's done
    print("All done!")
    
    return df

## Duplicate Data Detection and Treatment


In [ ]:
import pandas as pd

# Load your dataset (replace 'your_data.csv' with your actual file)
# df = pd.read_csv('your_data.csv')

# Function to detect duplicate rows
def detect_duplicates(dataframe):
    duplicate_rows = dataframe[dataframe.duplicated()]
    return duplicate_rows

# Function to remove duplicate rows
def remove_duplicates(dataframe):
    dataframe_no_duplicates = dataframe.drop_duplicates()
    return dataframe_no_duplicates

# Example usage
# Display duplicate rows
duplicate_rows = detect_duplicates(df)
print("Duplicate Rows:")
print(duplicate_rows)

# Remove duplicate rows
df_no_duplicates = remove_duplicates(df)
print("\nDataFrame after Removing Duplicates:")
print(df_no_duplicates)